In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import glob

import cv2
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.signal import convolve2d

In [2]:
%matplotlib inline

1. Edge detection
2. Trace all images for each pixel
3. draw lines in hough space
4. apply threshold to hough space
5. pick (a,b) coordinates in hough space

In [3]:
HOUGH_SPACE_SIZE = 512
LINE_MIN_LENGTH = 64
NON_MAX_SUPPRESSION_DIST = 5

NUM_SINUSOIDS = 40

# TODO: pick the required threshold
LINE_EDGE_THRESHOLD = 80

In [4]:
def imshow_with_colorbar(index, img):
    plt.figure(index)
    ax = plt.subplot(111)
    im = ax.imshow(img)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)

def convolve(img, kernel):
    # TODO:
    # write convolve operation
    # (the output array may have negative values)
    # for result comparison:
    # return convolve2d(img, kernel, mode='valid')
    # return np.sum(np.multiply(img, kernel))
    kernel_h, kernel_w = kernel.shape
    img_h, img_w = img.shape
    img_h = img_h - kernel_h + 1
    img_w = img_w - kernel_h + 1
    new_img = np.zeros((img_h, img_w))
    for i in range(img_h):
        for j in range(img_w):
            new_img[i][j] = np.sum(img[i:i+kernel_h, j:j+kernel_h]*kernel)
    return new_img

def threshold(img, threshold_value):
    # TODO:
    # write threshold operation
    # funciton should return numpy array with 1 in pixels that are higher or equal to threshold, otherwise - zero

    # img_array = img.reshape(-1)
    # threshold_array = np.array([thres(val, threshold_value) for val in img_array])
    # return threshold_array
    
    img_copy = np.copy(img)
    h,w = img_copy.shape
    img_array = img_copy.reshape(-1)
    # threshold_array = np.array([thres(val, threshold_value) for val in img_array])
    threshold_array = np.array([1 if pix_ >= threshold_value else 0 for pix_ in img_array])
    threshold_array = threshold_array.reshape(h,w)
    return threshold_array
    # return (img >= threshold_value).astype(np.uint8)

def draw_line_in_hough_space(h_space, y, x):
    # TODO:
    # I propose to use polar coordinates instead of proposed y = a*x + b.
    # They are way easier for implementation.
    # The idea is the same. Short documentation: https://docs.opencv.org/3.4/d9/db0/tutorial_hough_lines.html

    for angle_index in range(HOUGH_SPACE_SIZE):
        angle = angle_index / HOUGH_SPACE_SIZE * (2 * math.pi)
        r_index = math.ceil(math.cos(angle)*x + math.sin(angle)*y)

        if r_index <= 0:
          continue
        
        h_space[r_index, angle_index] += 1

def count_lines(img, is_debug):
    if is_debug: imshow_with_colorbar(1, img)

    # TODO:
    # pick the kernel for convolve operation, you need to find edges
    k = np.array([[0,-1,0], [-1,4,-1],[0,-1,0]])
    # k = np.array([[-1,-1,-1], [-1,8,-1],[-1,-1,-1]])
    # k = np.array([[1,1,1], [1,-8,1],[1,1,1]])
    # k = np.array([[0,-1,0], [-1,8,-1],[0,-1,0]])

    # calculate convolve operation
    img_c = convolve(img, k)
    if is_debug: imshow_with_colorbar(2, img_c)

    # TODO:
    # apply thresholding (the result of the threshold should be array with zeros and ones)
    img_thr = threshold(img_c, LINE_EDGE_THRESHOLD) # LINE_EDGE_THRESHOLD
    if is_debug: imshow_with_colorbar(3, img_thr)

    h_space = np.zeros((HOUGH_SPACE_SIZE, HOUGH_SPACE_SIZE), dtype=np.int)

    # for each coordinate ...
    for y in range(img_thr.shape[0]):
        for x in range(img_thr.shape[1]):
            # if there is edge ...
            if img_thr[y,x] != 0:
                draw_line_in_hough_space(h_space, y, x)
    if is_debug: imshow_with_colorbar(4, h_space)

    # apply threshold for hough space.
    # TODO:
    # pick the threshold to cut-off smaller lines
    # h_space_mask = threshold(h_space, ...)
    h_space_mask = threshold(h_space, NUM_SINUSOIDS)

    if is_debug: imshow_with_colorbar(5, h_space_mask)

    # get indices of non-zero elements
    y_arr, x_arr = np.nonzero(h_space_mask)

    # calculate the lines number,
    # here is simple non maximum suppresion algorithm.
    # it counts only one point in the distance NON_MAX_SUPPRESSION_DIST
    lines_num = 0
    for i in range(len(y_arr)):
        has_neighbour = False
        for j in range(i):
            yi, xi = y_arr[i], x_arr[i]
            yj, xj = y_arr[j], x_arr[j]
            dy = abs(yi - yj)
            dx = abs(xi - xj)
            if dy <= NON_MAX_SUPPRESSION_DIST and \
                (dx <= NON_MAX_SUPPRESSION_DIST or \
                 dx >= HOUGH_SPACE_SIZE - NON_MAX_SUPPRESSION_DIST):  # if x axis represents the angle, than check the distance if the points points that are near 0 degree (for example, distance between 1 deg. and 359 deg. is 2 deg.)
                has_neighbour = True
                break

        if not has_neighbour:
            lines_num += 1

    if is_debug: print('lines number: %d' % lines_num); plt.show()
    return lines_num

In [5]:
if __name__ == '__main__':
    fpath_arr = glob.glob('./images/*.png')
    fpath_arr.sort()

    print(fpath_arr)

    for img_fpath in fpath_arr:
        img = cv2.imread(img_fpath, cv2.IMREAD_GRAYSCALE)
        print('Number of lines: %d' % count_lines(img, False))

['./images/test_00.png', './images/test_01.png', './images/test_02.png', './images/test_03.png', './images/test_04.png', './images/test_05.png', './images/test_06.png', './images/test_07.png', './images/test_08.png']
Number of lines: 1
Number of lines: 2
Number of lines: 3
Number of lines: 4
Number of lines: 4
Number of lines: 4
Number of lines: 4
Number of lines: 4
Number of lines: 4
